# Model exploration

## Todo

- add more metrics
    - mutual info score
- multi variate output
- table of results
- Rhys: Compare the functional form of empirical models to that of LSMs, see where they differ
    - multivariate functional form
- 


In [ ]:
import numpy as np
import pylab as pl
import xray
import pandas as pd

from numbers import Number
from collections import OrderedDict

In [ ]:
pd.options.display.max_rows = 8

In [ ]:
%pylab inline
pl.rcParams['figure.figsize'] = (12.0, 3)
from IPython.display import display, HTML

In [ ]:
#import mpld3
#mpld3.enable_notebook()

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split

In [ ]:
from sklearn.linear_model import LinearRegression, Perceptron, SGDRegressor, LogisticRegression, PassiveAggressiveRegressor
from sklearn.svm import SVR, NuSVR  #, LinearSVR
# from sklearn.neural_network import MultilayerPerceptronRegressor # This is from a pull request: https://github.com/scikit-learn/scikit-learn/pull/3939
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
met_vars = ['SWdown', 'Tair', 'LWdown', 'Wind', 'Rainf', 'PSurf', 'Qair']
met_data = xray.open_dataset('/home/naught101/phd/data/PALS/datasets/met/TumbaFluxnet.1.4_met.nc')
met_df = met_data.to_dataframe().reset_index(['x','y','z']).ix[:, met_vars]

flux_vars = ['Qh', 'Qle', 'Rnet', 'NEE']
flux_data = xray.open_dataset('/home/naught101/phd/data/PALS/datasets/flux/TumbaFluxnet.1.4_flux.nc')
flux_df = flux_data.to_dataframe().reset_index(['x','y']).ix[:, flux_vars]


In [ ]:
flux_df[0:2]

In [ ]:
(met_df[0:3]).shift()

In [ ]:
import time

def timeit(f):
    def timed(*args, **kw):
        ts = time.time()
        result = f(*args, **kw)
        te = time.time()
        #print(f.__name__, 'took: {:2.4f} sec'.format(te-ts))        
        return (result, te-ts)
    return timed

In [ ]:
@timeit
def fit_pipeline(pipe, X, Y):
    pipe.fit(X, Y)
    
    
@timeit
def get_pipeline_prediction(pipe, X):    
    return(pipe.predict(X))


def get_pipeline_name(pipe):
    return(', '.join(pipe.named_steps.keys()))

In [ ]:
# METRICS
metrics = OrderedDict()

def rmse(obs, pred):
    return(np.sqrt(np.mean((obs - pred)**2)))
metrics.update({'rmse': rmse})

def nme(obs, pred):
    '''PLUMBER normalised mean error'''
    return(np.sum(np.abs(obs - pred))/np.sum(np.abs(obs - np.mean(obs))))
metrics.update({'nme': nme})

def mbe(obs, pred):
    '''PLUMBER mean bias error'''
    return(np.sum(pred - obs)/len(obs))
metrics.update({'mbe': mbe})

def sd_diff(obs, pred):
    '''PLUMBER standard deviation difference'''
    return(abs(1 - np.std(pred)/np.std(obs)))
metrics.update({'sd_diff': sd_diff})

def corr(obs, pred):
    return(np.corrcoef(obs, pred)[0,1])
metrics.update({'corr': corr})

def extreme_5(obs, pred):
    '''PLUMBER 5th percentile difference'''
    return(np.abs(np.percentile(pred, 5) - np.percentile(obs, 5)))
metrics.update({'extreme_5': extreme_5})

def extreme_95(obs, pred):
    '''PLUMBER 95th percentile difference'''
    return(np.abs(np.percentile(pred, 95) - np.percentile(obs, 95)))
metrics.update({'extreme_95': extreme_95})

# TODO: None of these make sense
def skewness(obs, pred):
    '''PLUMBER skewness'''
    skewness = sum(((pred - np.mean(pred))/np.std(pred))**3)/len(pred)
    return(skewness)
metrics.update({'skewness': skewness})

def kurtosis(obs, pred):
    '''PLUMBER kurtosis'''
    kurtosis = (sum(((pred - np.mean(pred))/np.std(pred))**4) - 3)/len(pred)
    return(kurtosis)
metrics.update({'kurtosis': kurtosis})

def overlap(obs, pred):
    '''PLUMBER overlap'''
    overlap = np.sum(np.minimum(obs,pred))
    return(overlap)
metrics.update({'overlap': overlap})


In [ ]:
a, b = (np.random.rand(100), np.random.rand(100))
[print(n,':', "{:.3f}".format(m(a,b))) for (n, m) in metrics.items()]


In [ ]:
def test_pipeline(pipe, X=met_df, Y=flux_df, y_var='Qh', name=None):
    if name is None:
        name = get_pipeline_name(pipe)

    Y = np.array(Y[y_var])
    
    
    train_len = (7*len(X)//10)
    
    # X_train, X_validate, Y_train, Y_validate = train_test_split(X, Y, train_size=0.7, random_state=0)
    X_train = X[:train_len]
    X_validate = X[train_len:]
    Y_train = Y[:train_len]
    Y_validate = Y[train_len:]
    
    metadata = OrderedDict({'name': name})  #, 'model': pipe.named_steps})
    
    metadata['t_fit'] = fit_pipeline(pipe, X_train, Y_train)[1]
    (Y_pred, metadata['t_pred']) = get_pipeline_prediction(pipe, X_validate)
    
    #print(name)
    #[print('\t', k, ': ', v) for (k, v) in pipe.steps]
    print("debug_shape: ", Y_pred.shape)
    print('---')
    if len(Y_pred.shape) > 1:
        Y_pred = Y_pred[:,0]
    
    for (n, m) in metrics.items():
        metadata[n] = m(Y_pred, Y_validate)
    metadata['corr'] = np.corrcoef(Y_pred, Y_validate)[0,1]
    [print('{:>10}:'.format(k), '{:.3f}'.format(v) if isinstance(v, Number) else v) for (k,v) in metadata.items()]
    
    # Sample plot
    plot_data = pd.DataFrame({y_var+'_obs': Y_validate, y_var+'_pred': Y_pred})
    
    # week 7 raw
    pl.plot(plot_data[(70*48):(77*48)])
    pl.legend(plot_data.columns)
    pl.show()
    
    # fornightly rolling mean
    pl.plot(pd.rolling_mean(plot_data, window=14*48))
    pl.legend(plot_data.columns)
    pl.show()
    
    #daily cycle
    pl.plot(plot_data.groupby(np.mod(plot_data.index, 48)).mean())
    pl.legend(plot_data.columns)
    pl.show()
    
    
    return(metadata)
    

In [ ]:
metadata = []

## Linear regression

- insensitive to scaling or PCA

In [ ]:
pipe = make_pipeline(LinearRegression())
metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(StandardScaler(), LinearRegression())
#metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(PCA(), LinearRegression())
#metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(StandardScaler(), PCA(), LinearRegression())
#metadata.append(test_pipeline(pipe))

## Polynomial regression

- Only a slight improvement
    - because non-linearities are localised?

In [ ]:
pipe = make_pipeline(PolynomialFeatures(2), LinearRegression())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(PolynomialFeatures(5), LinearRegression())
metadata.append(test_pipeline(pipe))

In [ ]:
met_df_with_lag = pd.concat([met_df, met_df.diff()], axis=1).dropna()
met_df_with_lag.shape

In [ ]:
np.linalg.matrix_rank(np.array(met_df_with_lag[:40000]))

In [ ]:
flux_df.shape

In [ ]:
flux_df[1:40001].shape

In [ ]:
pipe = make_pipeline(LinearRegression())
metadata.append(test_pipeline(pipe, X=met_df_with_lag[:40000], Y=flux_df[1:40001]))

## SGD

- very sensitive to scaling. Not sensitive to PCA

In [ ]:
#pipe = make_pipeline(SGDRegressor())
#metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), SGDRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(PCA(), SGDRegressor())
#metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(StandardScaler(), PCA(), SGDRegressor())
#metadata.append(test_pipeline(pipe))

In [ ]:
#test_model("LogisticRegression", LogisticRegression())

In [ ]:
#test_model("PassiveAggressiveRegressor", PassiveAggressiveRegressor())

## Support Vector Machines

- Sensitive to scaling, not to PCA

In [ ]:
#pipe = make_pipeline(SVR())
#metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), SVR())
metadata.append(test_pipeline(pipe))

In [ ]:
#pipe = make_pipeline(StandardScaler(), PCA(), SVR())
#metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), SVR(kernel='poly'))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ', poly kernel'))

## Multilayer Perceptron

In [ ]:
pipe = make_pipeline(MultilayerPerceptronRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor())
metadata.append(test_pipeline(pipe))  

In [ ]:
pipe = make_pipeline(PCA(), MultilayerPerceptronRegressor())
metadata.append(test_pipeline(pipe))  

In [ ]:
pipe = make_pipeline(StandardScaler(), PCA(), MultilayerPerceptronRegressor())
metadata.append(test_pipeline(pipe))           

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor(activation='logistic'))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ', logisitic'))

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor(hidden_layer_sizes=(20,20,20,)))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ", [20,20,20]"))

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor(hidden_layer_sizes=(10,10,)))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ", [10,10]"))

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor(hidden_layer_sizes=(10,30,)))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ", [10,30]"))

In [ ]:
pipe = make_pipeline(StandardScaler(), MultilayerPerceptronRegressor(hidden_layer_sizes=(20,20,)))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ", [20,20]"))

## K-nearest neighbours 

- Not sensitive to scaling or PCA

In [ ]:
pipe = make_pipeline(KNeighborsRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
mpld3.display()

In [ ]:
pipe = make_pipeline(StandardScaler(), KNeighborsRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(PCA(), KNeighborsRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), KNeighborsRegressor(n_neighbors=1000))
metadata.append(test_pipeline(pipe, get_pipeline_name(pipe) + ", 1000 neighbours"))

## Decision Trees

In [ ]:
pipe = make_pipeline(DecisionTreeRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(ExtraTreesRegressor())
metadata.append(test_pipeline(pipe))

In [ ]:
pipe = make_pipeline(StandardScaler(), PCA(), ExtraTreesRegressor())
metadata.append(test_pipeline(pipe))

# Metadata results

In [ ]:
metadata_df = pd.DataFrame(metadata).set_index('name')

In [ ]:
metadata_df

# Blah

In [ ]:
import math
import numpy
import scipy
from scipy.stats import gaussian_kde
from scipy.integrate import dblquad

def mutual_info(x,y):
    # Constants
    MIN_DOUBLE = 4.9406564584124654e-324 
                        # The minimum size of a Float64; used here to prevent the
                        #  logarithmic function from hitting its undefined region
                        #  at its asymptote of 0.
    INF = float('inf')  # The floating-point representation for "infinity"

    # x and y are previously defined as collections of 
    # floating point values with the same length

    # Kernel estimation
    gkde_x = gaussian_kde(x)
    gkde_y = gaussian_kde(y)
    gkde_xy = gaussian_kde([x,y])

    mutual_info = lambda a,b: gkde_xy([a,b]) * \
               math.log((gkde_xy([a,b]) / (gkde_x(a) * gkde_y(b))) + MIN_DOUBLE)

    # Compute MI(X,Y)
    (minfo_xy, err_xy) = dblquad(mutual_info, -INF, INF, lambda a: 0, lambda a: INF)

    print('minfo_xy = ', minfo_xy)


In [ ]:
mutual_info(met_df.SWdown, flux_df.Qh)

In [ ]:
met_df.corr()

In [ ]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
import numpy as np

iris = datasets.load_iris()
iris.data[0:30, 1] = np.random.rand(30) * iris.data[0:30, 1]
X_train = iris.data[0:100, :2]
Y_train = iris.data[0:100, 3]
X_test = iris.data[100:150, :2]
Y_test = iris.data[100:150, 3]

scaler = StandardScaler()
model = LinearRegression(normalize=True)

model.fit(X_train, Y_train)
pred = model.predict(X_test)

print('RMSE    raw: ', (np.mean((pred-Y_test)**2))**0.5)

model.fit(scaler.fit_transform(X_train), Y_train)
pred = model.predict(scaler.transform(X_test))

print('RMSE scaled: ', (np.mean((pred-Y_test)**2))**0.5)


In [ ]:
plot(pred, Y_test)

In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
X, y = np.arange(10).reshape((5, 2)), range(5)
print(X)
print(list(y))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

print(X_train)
print(y_train)
print(X_test)
print(y_test)

